In [1]:
%autosave 0

Autosave disabled


In [1]:
import tensorflow as tf
from tensorflow import keras

In [14]:
model = keras.models.load_model("./clothing-model-v4.h5")

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-05 15:35:13.860946: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-05 15:35:13.861339: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
model

In [16]:
tf.saved_model.save(model, 'clothing-model')

INFO:tensorflow:Assets written to: clothing-model/assets


INFO:tensorflow:Assets written to: clothing-model/assets


In [17]:
!saved_model_cli show --dir clothing-model --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Concrete Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          input_8: Tensor

#### Model local deployment

`docker run -it --rm \`
  `-p 8500:8500 \`
  `-v $(pwd)/clothing-model:/models/clothing-model/1 \`
  `-e MODEL_NAME="clothing-model" \`
  `tensorflow/serving:2.7.0`

throws an error:

Status: Downloaded newer image for tensorflow/serving:2.7.0
[libprotobuf FATAL external/com_google_protobuf/src/google/protobuf/generated_message_reflection.cc:2345] CHECK failed: file != nullptr: 
terminate called after throwing an instance of 'google::protobuf::FatalException'
  what():  CHECK failed: file != nullptr: 
qemu: uncaught target signal 6 (Aborted) - core dumped
/usr/bin/tf_serving_entrypoint.sh: line 3:     8 Aborted                 tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} "$@"


Solution:
`docker pull emacski/tensorflow-serving:latest-linux_arm64`

`docker run -it --rm \`
  `-p 8500:8500 \`
  `-v $(pwd)/clothing-model:/models/clothing-model/1 \`
  `-e MODEL_NAME="clothing-model" \`
  `emacski/tensorflow-serving:latest-linux_arm64`





In [22]:
#!pip install grpcio==1.42.0 tensorflow-serving-api==2.6.0

#!pip install grpcio==1.42.0 tensorflow-serving-api-python3

!pip3 install --no-deps tensorflow-serving-api

In [23]:
!pip install keras-image-helper

In [13]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [2]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
from keras_image_helper import create_preprocessor

In [4]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [5]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [8]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [9]:
preds = pb_response.outputs['dense_7'].float_val

In [10]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [11]:
dict(zip(classes, preds))

{'dress': -1.8682887554168701,
 'hat': -4.761242389678955,
 'longsleeve': -2.3169844150543213,
 'outwear': -1.0625687837600708,
 'pants': 9.887155532836914,
 'shirt': -2.8124337196350098,
 'shoes': -3.6662817001342773,
 'shorts': 3.2003633975982666,
 'skirt': -2.6023409366607666,
 't-shirt': -4.835043907165527}

In [ ]:
https://github.com/alexeygrigorev/mlbookcamp-code/releases/download/chapter7-model/xception_v4_large_08_0.894.h5